In [46]:
import pickle
# import streamlit as st

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import FAISS

In [47]:
loader = UnstructuredURLLoader(urls=["https://www.moneycontrol.com/news/business/markets/gold-climbs-as-cooling-us-inflation-bolsters-fed-rate-cut-bets-12723150.html"])
    # main_placeholder.text("Data Loading Started . . .")
data = loader.load()

In [48]:
text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n','.',','],
        chunk_size = 1000
    )
docs = text_splitter.split_documents(data)

In [49]:
import pprint
print(len(docs))

16


In [52]:
embeddings = GooglePalmEmbeddings()
vectorstore_genai = FAISS.from_documents(docs, embeddings) 

In [53]:
print(vectorstore_genai.index.ntotal)

16


In [54]:
query = 'what is the price of silver per ounce?'
docs = vectorstore_genai.similarity_search(query)

In [55]:
print(docs[1].page_content)

Spot gold was 0.5% higher at $2,369.49 per ounce by 1449 GMT. U.S. gold futures for June delivery rose 0.6% to $2,374.60 per ounce.

Story continues below Advertisement

Remove Ad

The consumer price index data "could be an early indication that over time inflation will cool and the Fed will make its first interest rate cut," said Phillip Streible, chief market strategist at Blue Line Futures.

U.S. CPI rose 0.3% last month after advancing 0.4% in March and February, suggesting that inflation resumed its downward trend at the start of the second quarter in a boost to financial market expectations for a September interest rate cut.

Economists polled by Reuters had forecast the CPI rising 0.4% on the month and advancing 3.4% year-on-year.

Related stories

Gold prices steady as focus turns to US Fed minutes

This gold rally is made in China

World Street | Gold hits record high, Crypto rally, Jamie Dimon's succession plans and more


In [56]:
vectorstore_genai.save_local('faiss_index')

In [68]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=do)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python**

* **Easy to learn:** Python is a very easy language to learn, even for beginners. The syntax is simple and straightforward, and there are a lot of resources available to help you get started.
* **Versatile:** Python can be used for a wide variety of tasks, including web development, data science, and machine learning. It's also a good language for beginners to learn because it can be used for so many different things.
* **Highly extensible:** Python is a very extensible language, which means that you can add new features and functionality to it very easily. This makes it a good choice for projects that require a lot of customization.
* **Active community:** Python has a large and active community of developers, which means that there is a lot of support available if you run into any problems. There are also a lot of resources available, such as books, tutorials, and online forums.

**Cons of Python**

* **Slow:** Python is not as fast as some other programming langu

In [60]:
new_faiss_index = FAISS.load_local('faiss_index', embeddings, allow_dangerous_deserialization=True)


In [64]:
query = 'what is the price of silver per ounce?'
docs = new_faiss_index.similarity_search(query)
print(docs)

[Document(page_content="Story continues below Advertisement\n\nRemove Ad\n\nSpot silver rose 1.2% to $28.93 per ounce, palladium gained 1.2% to $989.45 and platinum climbed 1.7% to $1,048.40, hitting a near one-year high.\n\nReuters\n\nTags:\n\n#Gold\n\n#US gold futures\n\n#US inflation data\n\nfirst published: May 15, 2024 09:18 pm\n\nDiscover the latest business news, Sensex, and Nifty updates. Obtain Personal Finance insights, tax queries, and expert opinions on Moneycontrol or download the Moneycontrol App to stay updated!\n\nAdvertisement\n\nRemove Ad\n\nTrending news\n\n'Europe, I don’t want to marry your men': This investor missed a flight, spent Rs 4 lakh waiting for Schengen visa\n\nCyclone Remal to reach Bengal today: How cyclonic storms get their names\n\n'I'm not a typo campaign': People with autocorrected names approach tech companies to fix 'bias'\n\n'Smoothest, safest and cleanest': Irish travellers share video on Delhi Metro, quash misconceptions\n\nGoogle marks 2024 IP

In [84]:
from langchain.chains import RetrievalQA
# from langchain_community.chat_models import google_palm

# palm_client = google_palm.ChatGooglePalm(google_api_key="YOUR_GOOGLE_PALM_API_KEY")

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=new_faiss_index.as_retriever(search_kwargs={"k": 100}),
)

In [85]:
print(qa.invoke(query))

{'query': 'what is the price of silver per ounce?', 'result': 'The current price of silver is $28.93 per ounce.'}
